In [24]:
%%capture
%load_ext autoreload
%autoreload 1

In [25]:
import pandas as pd
import numpy as np
from pipeliner.recommendations.transformer import (
    UserItemMatrixTransformerPandas,
    SimilarityTransformerPandas,
)
from pipeliner.recommendations.recommender import ItemBasedRecommender

In [26]:
data_types = {"user_id": str, "item_id": str, "rating": np.float64}
user_item_ratings = pd.read_csv("../tests/test_data/user_item_ratings.csv", dtype=data_types)
user_item_ratings.head(5)

,user_id,item_id,rating
0,U1000,I1063,0.36
1,U1001,I1062,0.79
2,U1001,I1099,0.43
3,U1002,I1043,0.64
4,U1002,I1013,0.43


In [27]:
user_item_matrix_transformer = UserItemMatrixTransformerPandas()
user_item_matrix = user_item_matrix_transformer.transform(user_item_ratings)
user_item_matrix.head(5)

item_id,I1003,I1006,I1010,I1013,I1016,I1017,I1019,I1020,I1021,I1025,...,I1074,I1075,I1076,I1077,I1082,I1091,I1093,I1097,I1098,I1099
user_id,,,,,,,,,,,,,,,,,,,,,
U1000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
U1001,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.43
U1002,0.00,0.0,0.0,0.43,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
U1003,0.00,0.0,0.5,0.00,0.0,0.0,0.0,0.0,0.00,0.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
U1004,0.79,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.71,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [28]:
similarity_matrix_transformer =  SimilarityTransformerPandas(kind="item", metric="cosine", normalise=True)
similarity_matrix = similarity_matrix_transformer.transform(user_item_matrix)
similarity_matrix.head(5)

item_id,I1003,I1006,I1010,I1013,I1016,I1017,I1019,I1020,I1021,I1025,...,I1074,I1075,I1076,I1077,I1082,I1091,I1093,I1097,I1098,I1099
item_id,,,,,,,,,,,,,,,,,,,,,
I1003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.762141,0.647411,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.762141,0.0,0.0
I1006,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,1.0,1.0,0.000000,0.0,0.0
I1010,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
I1013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.758185,0.0,0.0,0.0,0.000000,0.0,0.0
I1016,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,1.0,0.0


Generate recommendations based on item similarity

In [29]:
item_id = "I1003"

item_recommendations = (
    similarity_matrix[item_id]
    .drop(item_id, errors="ignore")
    .sort_values(ascending=False)
)
# double check the item isn't in the recommendations
# this should be an empty series
assert item_id not in item_recommendations.index

item_recs_1 = item_recommendations.head(5).index.to_list()
item_recs_1

['I1053', 'I1058', 'I1097', 'I1020', 'I1067']

results should be the same with the library

In [30]:
rec = ItemBasedRecommender(5)

rec.fit(similarity_matrix)
item_recs_2 = rec.predict([item_id]).tolist()[0]
print(item_recs_2)

assert set(item_recs_1) == set(item_recs_2)

['I1053', 'I1058', 'I1097', 'I1020', 'I1067']


Generate recommendations based on item similarity, excluding user's past items

In [31]:
user_id = "U1002"

single_user_matrix = user_item_matrix.loc[user_id]
user_rated_items = single_user_matrix[single_user_matrix > 0].sort_values(
    ascending=False
)
print("user_rated_items", user_rated_items.index.to_list())

user_item_recommendations = (
    similarity_matrix[item_id]
    .drop([item_id] + user_rated_items.index.to_list(), errors="ignore")
    .sort_values(ascending=False)
)

# double check the user hasn't rated the items
# this should be an empty series
assert (
    set(user_rated_items.index.to_list()).intersection(
        set(user_item_recommendations.index.to_list())
    )
    == set()
)

user_item_recs_1 = user_item_recommendations.head(5).index.to_list()
user_item_recs_1

user_rated_items ['I1043', 'I1051', 'I1013']


['I1097', 'I1058', 'I1020', 'I1053', 'I1040']

In [32]:
rec2 = ItemBasedRecommender(5)

rec2.fit((similarity_matrix, user_item_matrix))
user_item_recs_2 = rec2.predict([(item_id, user_id)]).tolist()[0]

assert set(user_item_recs_1) == set(user_item_recs_2)

user_item_recs_2

['I1097', 'I1058', 'I1020', 'I1053', 'I1040']